In [1]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import openai
import os
import emoji
import time
from datetime import datetime
import holidays

Python-dotenv could not parse statement starting at line 1


In [2]:
openai.api_key  = os.getenv('OPENAI_API_KEY')
df = pd.read_csv("psa.csv", sep = ",")
df = df.drop(['Date', 'Campaign ID','Excluded List'], axis = 1)

# pd.set_option('display.max_columns', 100)
# pd.set_option('display.max_row', 100)

In [3]:
df.shape

(484, 21)

# Time Function

In [4]:
def map_time_to_range(time_str):
    time_obj = datetime.strptime(time_str, '%I:%M %p')  # Convert string to datetime object
    hour = time_obj.hour

    if 0 <= hour < 2.59:
        return '0-2'
    elif 3 <= hour < 5.59:
        return '3-5'
    elif 6 <= hour < 8.59:
        return '6-8'
    elif 9 <= hour < 11.59:
        return '9-11'
    elif 12 <= hour < 14.59:
        return '12-14'
    elif 15 <= hour < 17.59:
        return '15-17'
    elif 18 <= hour < 20.59:
        return '18-20'
    elif 21 <= hour < 23.59:
        return '21-23'

In [5]:
def map_time_to_range_enm(time_str):
    time_obj = datetime.strptime(time_str, '%I:%M %p')  # Convert string to datetime object
    hour = time_obj.hour

    if 0 <= hour < 3.59:
        return '0-3'
    elif 4 <= hour < 7.59:
        return '4-7'
    elif 8 <= hour < 11.59:
        return '8-11'
    elif 12 <= hour < 15.59:
        return '12-15'
    elif 16 <= hour < 19.59:
        return '16-19'
    elif 20 <= hour < 23.59:
        return '20-23'

In [6]:
def map_time_to_range_emp(time_str):
    time_obj = datetime.strptime(time_str, '%I:%M %p')  # Convert string to datetime object
    hour = time_obj.hour

    if 0 <= hour < 5.59:
        return '0-5'
    elif 6 <= hour < 11.59:
        return '6-11'
    elif 12 <= hour < 17.59:
        return '12-17'
    elif 18 <= hour < 23.59:
        return '18-23'

In [7]:
df['time_period_8'] = df['Send Time'].apply(map_time_to_range)
df['time_period_6'] = df['Send Time'].apply(map_time_to_range_enm)
df['time_period_4'] = df['Send Time'].apply(map_time_to_range_emp)

In [8]:
df.shape

(484, 24)

# Emoji Function

In [10]:
def is_emojii(text):
    if emoji.emoji_count(text) > 0 :
        return 1
    return 0

In [11]:
df['is_Emoji'] = df['Subject'].apply(is_emojii)

In [12]:
df.shape

(484, 25)

# Lenght Function

In [13]:
def panjang(x) :
    if len(x) <= 50 :
        return 1
    return 0

In [14]:
df['subject_length_below_50'] = df['Subject'].apply(panjang)

In [15]:
df.shape

(484, 26)

# Month Function

In [16]:
def extract_month(month):
    our_month = datetime.strptime(month, "%Y-%m-%d")
    return our_month.month

In [17]:
df['Month'] = df['Send Date'].apply(extract_month)

In [18]:
df.shape

(484, 27)

# Holiday Function

In [19]:
def is_holiday(date_str):
    try:
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")
        indonesian_holidays = holidays.Indonesia(years=date_obj.year)
        return int(date_obj in indonesian_holidays)
    except:
        return 0

In [20]:
df['is_holiday'] = df['Send Date'].apply(is_holiday)

In [21]:
df.shape

(484, 28)

# Exclude Function

In [22]:
def keyword_ex(campaign_name):
    keywords = ["voucher", "testing", "test", "birthday", "testing", "lapsed", "Voucher", "Testing", "Test", "Birthday", "Lapsed", "uji", "coba"]
    
    for keyword in keywords:
        if keyword in campaign_name:
            return 1
    
    return 0

In [23]:
df['is_Exclude'] = df['Campaign Name'].apply(keyword_ex)

In [24]:
df.shape

(484, 29)

# Exclude Certain Campaign

In [29]:
df.shape

(484, 29)

In [31]:
df_exclude = df[df['is_Exclude'] != 1]
test_ex.shape

(351, 29)

# Predict CTA

In [25]:
def predict_cta(text):
    prompt = f"Classify whether the following text contains a Call to Action (CTA) or not: {text}\n Please only respond with '1' for 'Yes' and '0' for 'No'."
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    
    return response.choices[0].message.content

In [35]:
df_exclude['is_CTA'] = df_exclude['Subject'].apply(predict_cta)

C:\Users\wvinc\AppData\Local\Temp\ipykernel_22380\1697465428.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_exclude['is_CTA'] = df_exclude['Subject'].apply(predict_cta)


In [36]:
df_exclude.shape

(351, 30)

# Predict Limited

In [37]:
def predict_limited_offer(text):
    prompt = f"Determine whether the following text represents a limited offer promotion: {text}\n Please only respond with '1' for 'Yes' and '0' for 'No'."
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", "content": prompt}
      ]
    )
    
    return response.choices[0].message.content

In [49]:
df_exclude['is_Limited'] = df_exclude['Subject'].apply(predict_limited_offer)

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

In [42]:
df_exclude.shape

(351, 30)

In [50]:
df_merge = df_exclude.merge(df[['Campaign Name', 'is_Limited']], on='Campaign Name', how='left')

In [51]:
df_merge.shape

(353, 31)

In [112]:
df_merge['Tag'] = df_merge['Campaign Name'].str.extract(r'_(EB|NB)')

# Column Without EB & NB

In [114]:
df_split = df_merge.copy()

In [115]:
df_split.shape

(353, 32)

In [116]:
df_split['Campaign Name2'] = df_split['Campaign Name'].str.replace('_NB', '').str.replace('_EB', '')

In [108]:
df_split.head()

,Campaign Name,Send Date,Send Time,Total Recipients,Open Rate,Click Rate,Unsubscribe Rate,Spam Complaints Rate,Total Opens,Unique Opens,...,time_period_6,time_period_4,is_Emoji,subject_length_below_50,Month,is_holiday,is_Exclude,is_CTA,is_Limited,Campaign Name2
0,20230815_EDM_MAA SPORTS_PSA DF_Brand Takeover ...,2023-08-15,11:00 AM,10215,0.210908,0.003231,0.001077,0.000098,2578,2154,...,8-11,6-11,1,1,8,0,0,1,0,20230815_EDM_MAA SPORTS_PSA DF_Brand Takeover ...
1,20230815_EDM_MAA SPORTS_PSA DF_Brand Takeover ...,2023-08-15,11:00 AM,105049,0.505187,0.001629,0.001658,0.000076,62423,53030,...,8-11,6-11,1,1,8,0,0,0,0,20230815_EDM_MAA SPORTS_PSA DF_Brand Takeover ...
2,20230811_EDM_MAA SPORTS_PSA DF_PSA Splash Out!...,2023-08-11,04:00 PM,10873,0.215324,0.003771,0.000368,0.000184,2820,2341,...,16-19,12-17,0,1,8,0,0,1,0,20230811_EDM_MAA SPORTS_PSA DF_PSA Splash Out!...
3,20230811_EDM_MAA SPORTS_PSA DF_PSA Splash Out!...,2023-08-11,04:00 PM,110426,0.540356,0.002058,0.001868,0.000163,70885,59598,...,16-19,12-17,0,1,8,0,0,0,0,20230811_EDM_MAA SPORTS_PSA DF_PSA Splash Out!...
4,20230730_EDM_MAA SPORTS_PSA DF_Brand takeover ...,2023-07-30,11:00 AM,8656,0.213634,0.004506,0.000347,0.000116,2328,1849,...,8-11,6-11,1,1,7,0,0,0,0,20230730_EDM_MAA SPORTS_PSA DF_Brand takeover ...


In [118]:
df_split.to_csv('DF_split_final.csv', index=False)

# Re-join CTA & Limited

In [120]:
df_split_drop = df_split.drop(['is_CTA', 'is_Limited'],axis = 1)

In [121]:
cta_only = pd.read_csv('cta_only.csv')

In [122]:
df_split_drop.shape

(353, 31)

In [125]:
df_split_drop = df_split_drop.merge(cta_only[['Subject', 'is_CTA', 'is_Limited']], on='Subject', how='left')

In [126]:
df_split_drop.shape

(353, 33)

In [127]:
df_split_drop.to_csv('DF.csv', index=False)